In [1]:
from selenium import webdriver
from requests import get
from bs4 import BeautifulSoup
import bs4
import pandas as pd
import numpy as np
from time import sleep
from random import randint
from time import time
from warnings import warn
from datetime import datetime
from datetime import timedelta
from pytz import timezone
from IPython.display import clear_output
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
import re
import os
import sys
import glob
import shutil
from pathlib import Path

In [2]:
# Settings for web driver and Proxy
chrome_driver_path = os.path.join(os.getcwd(),'chromedriver.exe')
# PROXY = "socks5://localhost:9050" # IP:PORT or HOST:PORT
# create a new Chrome session
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
# options.add_argument('--headless')
# options.add_argument('--proxy-server=%s' % PROXY)
driver = webdriver.Chrome(options=options)
driver.implicitly_wait(0)

In [3]:
movies = pd.read_csv('Movie_TXL.csv')
movies

,url,title,on_streaming_date,genre,certification,runtime,rating,content,Director,Director_link,...,cast1_link,cast2,cast2_link,cast3,cast3_link,Status,Original Language,Budget,Revenue,keyword
0,https://www.themoviedb.org//movie/10851,Revolver,2005-09-22,"Drama, Thriller, Crime, Mystery",15,111,64.0,Your mind will not accept a game this big.\nHo...,Guy Ritchie,https://www.themoviedb.org//person/956-guy-rit...,...,https://www.themoviedb.org//person/976-jason-s...,Ray Liotta,https://www.themoviedb.org//person/11477-ray-l...,Vincent Pastore,https://www.themoviedb.org//person/47774-vince...,Released,English,27000000.0,7164778.0,"cash,wrong diagnosis,foretelling,shaky camera ..."
1,https://www.themoviedb.org//movie/10759,The Comebacks,2007-10-19,Comedy,PG-13,107,49.0,Keep your eye on the ball.\nOut-of-luck coach ...,Tom Brady,https://www.themoviedb.org//person/66512-tom-b...,...,https://www.themoviedb.org//person/1101-carl-w...,David Koechner,https://www.themoviedb.org//person/28638-david...,Melora Hardin,https://www.themoviedb.org//person/404-melora-...,Released,English,15000000.0,13500000.0,"baseball,baseball bat,sports,coach"
2,https://www.themoviedb.org//movie/78698,Big Miracle,2012-02-10,"Adventure, Drama, Romance",PG,107,66.0,A family of whales trapped beneath the ice. On...,Ken Kwapis,https://www.themoviedb.org//person/29009-ken-k...,...,https://www.themoviedb.org//person/69597-drew-...,John Krasinski,https://www.themoviedb.org//person/17697-john-...,Kristen Bell,https://www.themoviedb.org//person/40462-krist...,Released,English,40000000.0,24719215.0,"whale,based on true story,alaska,reporter,afte..."
3,https://www.themoviedb.org//movie/9275,Death to Smoochy,2002-03-29,"Comedy, Crime, Drama, Thriller",R,109,59.0,Get ready for an unexpected hit.\nTells the st...,Danny DeVito,https://www.themoviedb.org//person/518-danny-d...,...,https://www.themoviedb.org//person/2157-robin-...,Edward Norton,https://www.themoviedb.org//person/819-edward-...,Catherine Keener,https://www.themoviedb.org//person/2229-cather...,Released,English,55000000.0,8382938.0,"corruption,success,irish mob,duringcreditsstin..."
4,https://www.themoviedb.org//movie/7350,The Bucket List,2007-12-25,"Drama, Comedy",PG-13,97,72.0,Find the joy.\nCorporate billionaire Edward Co...,Rob Reiner,https://www.themoviedb.org//person/3026-rob-re...,...,https://www.themoviedb.org//person/514-jack-ni...,Morgan Freeman,https://www.themoviedb.org//person/192-morgan-...,Sean Hayes,https://www.themoviedb.org//person/52601-sean-...,Released,English,45000000.0,175372502.0,"africa,himalaya mountain range,brain tumor,par..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3433,https://www.themoviedb.org//movie/339877,Loving Vincent,2017-10-06,"Animation, Drama, Mystery, History",C16,95,80.0,"The truth is, we cannot speak other than by ou...",DK Welchman,https://www.themoviedb.org//person/1466181-hug...,...,https://www.themoviedb.org//person/230680-doug...,Robert Gulaczyk,https://www.themoviedb.org//person/1702761-rob...,Eleanor Tomlinson,https://www.themoviedb.org//person/73357-elean...,Released,English,5500000.0,42187665.0,"painter,artist,letter,investigation,bullying,s..."
3434,https://www.themoviedb.org//movie/280391,Animal Kingdom: Let's Go Ape,2015-04-08,"Adventure, Comedy, Animation",U,100,53.0,These feet are made for walkin'\nBased on the ...,Jamel Debbouze,https://www.themoviedb.org//person/2408-jamel-...,...,https://www.themoviedb.org//person/2408-jamel-...,Arié Elmaleh,https://www.themoviedb.org//person/82302-arie-...,Mélissa Theuriau,https://www.themoviedb.org//person/1254992-mel...,Released,French,37580800.0,16294916.0,"animal species,evolution,humanity"
3435,https://www.themoviedb.org//movie/71676,Ghost Rider: Spirit of Vengeance,2011-12-10,"Action, Fantasy, Thriller",PG-13,96,50.0,He rides again.\nWhen the devil resurfaces wit...,Mark Neveldine,https://www.themoviedb.org//person/20193-mark-...,...,https://www.themoviedb.org//person/2

In [4]:
merged_list = movies[['Director_link','cast1_link','cast2_link','cast3_link']].values.ravel().tolist()
print(len(merged_list))
actor_list = list(set(merged_list))
len(actor_list)

13752


5672

In [5]:
item_list = []
for url in actor_list:
    print(actor_list.index(url))
    item_dict={}
    item_dict["url"]=url
    driver.get(url)
    # sleep(3)
    element= WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div[class="white_column"]')))
    html_of_interest=driver.execute_script('return arguments[0].innerHTML',element)
    soup_level1=BeautifulSoup(html_of_interest, 'lxml')
    title = soup_level1.select('div[class="title"]')
    item_dict["title"]=title[0].text.strip()
    biography = soup_level1.select('section[class="full_wrapper"]')[0].select('p')
    bio = ''
    for b in biography:
        bio = bio + ' ' + b.text.strip()
    item_dict["biography"]=bio[1:]
    domestic = soup_level1.select('ul[class="horizontal_media_list scroller"]')
    do = ''
    if len(domestic)==0:
        item_dict["domestic_movie"]='-'
        item_dict["number_domestic"]=0
    elif len(domestic[0].select('a[class="title"]'))<5:
        for i in range(len(domestic)):
            do = do + ', ' + domestic[0].select('a[class="title"]')[i].text.strip()
        item_dict["domestic_movie"]=do[2:]
        item_dict["number_domestic"]=len(domestic[0].select('a[class="title"]'))
    else:
        for i in range(5):
            do = do + ', ' + domestic[0].select('a[class="title"]')[i].text.strip()
        item_dict["domestic_movie"]=do[2:]
        item_dict["number_domestic"]=len(domestic[0].select('a[class="title"]'))
    years = soup_level1.select('table[class="card credits"]')[0].select('tr')[-1].select('td')[0]
    item_dict["launch_year"]=years.text.strip()
    element= WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div[class="grey_column"]')))
    html_of_interest=driver.execute_script('return arguments[0].innerHTML',element)
    soup_level2=BeautifulSoup(html_of_interest, 'lxml')
    facts = soup_level2.select('section[class="facts"]')[0].select('p')
    for f in facts[:-1]:
        item_dict[f.select('strong')[0].text.strip()] = f.text.strip()[len(f.select('strong')[0].text.strip())+1:]
    item_dict["Birthday"]=item_dict["Birthday"].strip()
    # print(item_dict)
    item_list.append(item_dict)
df=pd.DataFrame(item_list)
# print(df.columns)
df.head(100000)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


KeyboardInterrupt: 

In [ ]:
df.info()

In [ ]:
df['id'] = df['url'].str.extractall('(\d+)').groupby(level=0)[0].apply(''.join)
df

In [ ]:
df.to_csv('Cast.csv', index=False)
df

In [ ]:
movies['Director_link'] = movies['Director_link'].str.extractall('(\d+)').groupby(level=0)[0].apply(''.join)
movies['cast1_link'] = movies['cast1_link'].str.extractall('(\d+)').groupby(level=0)[0].apply(''.join)
movies['cast2_link'] = movies['cast2_link'].str.extractall('(\d+)').groupby(level=0)[0].apply(''.join)
movies['cast3_link'] = movies['cast3_link'].str.extractall('(\d+)').groupby(level=0)[0].apply(''.join)
movies

In [ ]:
movies = movies.rename(columns={'Director_link': 'Director_id','cast1_link': 'cast1_id','cast2_link': 'cast2_id','cast3_link': 'cast3_id'})

In [ ]:
movies.to_csv('Movie_model.csv', index=False)